In [1]:
import sys
sys.path.append('/home/harish/GitRepos/Smart_Helm/')
import pyxdf
import pandas as pd
from datetime import datetime, timedelta, date
from data_processing.raw_data_parser import *
from data_processing.fact_prep import *
import numpy as np

In [3]:
annotated_distractions_df = pd.read_csv("./test_files/annotated.csv")
annotated_distractions_df = annotated_distractions_df.rename(columns={"distraction_start": "start_lsl"})

In [ ]:
data_streams, header  = pyxdf.load_xdf("./test_files/Rytle_KZ_02.xdf")

## Goal: creation of the visual distraction dimension table

### TODO 1: (Extraction step) extraction of the annotated distractions from the Minio

### TODO 2: Transformation of the annotations 

#### step1: fetch the data from the minio:

#### final schema of the visual distraction dimension table
| name | datatype | is_nullable | value |
| -- | -- | -- | -- |
| start_lsl | float64 | True | 3838.634852 |
| end_lsl | float64 | True | 3844.528151 |
| is_visual | boolean | True | True |
| experiment_date | date | False | 2021-01-01 |
| participant_id | str | False | 'BC' |
| experiment_nr | int | False | 1 |
| experiment_location_id | str | False | 'CSL' |

##### METHOD
generate a column `next_lsltimestamp` to check where the annotated distractions fall in between. the from this point to the next 4 seconds (i. 4 * 30 hz 120 timestamps) a newcolumn with bool expression True/False created, 
* TODO1: a column  which specifies the annotation end time i.e from the coinciding start time to the end
* TODO2: in between the start to end a new column should genrated to give a bool expression say TRue or false

### TODO 3: Loading i.e populating into the db
   

In [35]:
etd_df, exp_dt = transform_etd("./test_files/Rytle_J_02.xdf", "Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")
experiment_location_id, participant_id, experiment_nr = get_experiment_details("Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")

In [ ]:
ele = "Rytle/2021/10/05/G/01/annotated_distractions/visual.csv"

In [ ]:
parent_path, child_path = ele.split("annotated_distractions")

In [ ]:
parent_path

In [ ]:
child_path


--> implement two functions 
--> first fn from the example input to intermediate output
--> second fn from intermediate to final 

#### we need to merge the above time_windows with the following conditions
1. they should be of the same type of distraction
2. the time windows should overlap    

example input:   

| start_lsl | end_lsl | is_visual |
| -- | -- | -- |
| 2 | 6 | True | 
| 12 | 16 | True |
| 6 | 10 | True | 
| 7 | 11 | True | 

 

example output:    

| start_lsl | end_lsl | is_visual |
| -- | -- | -- |
| 2 | 11 | True | 
| 12 | 16 | True | 

In [ ]:
example_input_df = pd.DataFrame(
    {
        "start_lsl": [2,12,6,7,13,20], 
        "end_lsl": [6,16,10,11, 17, 24]   
    }                            
)

In [ ]:
example_output_df = pd.DataFrame(
    {
        "start_lsl": [2,12,20], 
        "end_lsl": [11,16,24]
    }                            
)

In [ ]:
def transform(df):
    """The main goal of this function is to "reduce" the number of rows and "merge" all those time windows
    where the distraction is "continuous" e.g.
                start      end
    event 1:    10         14
    event 2:    12         16
    event 3:    20         24

    then the final output should be:
                start      end
    event 1:    10         16
    event 3:    20         24
    """

    # sort the whole data wrt `start_lsl` - so that we can use `rolling` in the next step
    df.sort_values(by="start_lsl", inplace=True)


    # identify the rows which belong to the same group - assign a `group_number` to each group present in the data
    group_number = 1
    df["group"] = group_number
    for df_window in df.rolling(window=2):
        # reset the index here for each window so that we can fetch the values below always 
        # using 0 for the first row and 1 for the second row
        df_window.reset_index(inplace=True)
        if len(df_window) > 1:
            interval_1 = pd.Interval(df_window.loc[0, "start_lsl"], df_window.loc[0, "end_lsl"], closed="both")
            interval_2 = pd.Interval(df_window.loc[1, "start_lsl"], df_window.loc[1, "end_lsl"], closed="both")
            if interval_1.overlaps(interval_2):
                # if overlap happens then get all the indices, using the `index` column of the original (parent or main) dataframe
                # and fill those rows in the main dataframe with the detected group number
                indices_of_groups = list(df_window["index"])
                df.loc[indices_of_groups, "group"] = group_number
            else:
                group_number = group_number + 1
                index_of_new_group = df_window.loc[1, "index"]
                df.loc[index_of_new_group, "group"] = group_number


    # aggregate using simple `groupby` - keeping the `start_lsl` as the `start_lsl` of the first (minimum) member of a group
    # and `end_lsl` of the last (maximum) member of the same group
    # do the same for all groups separately
    df = df.groupby(by="group").agg({"start_lsl": min, "end_lsl": max}).reset_index(drop=True)

    return df

In [ ]:
pd.testing.assert_frame_equal(transform2(example_input_df), example_output_df)

In [ ]:
#transform(annotated_distractions_df)

In [ ]:
visual_df = transform_visual_distraction(annotated_distractions_df, exp_dt)

In [ ]:
visual_df["start_lsl"].min()

## Goal2: integrate the above information finally to fact data table

In [ ]:
visual_df.loc[visual_df["start_lsl"] == visual_df["start_lsl"].min()]["start_lsl"].values[0]

In [6]:
get_visual_dis_lsl_tst_join(etd_df["lsl_timestamp"][10000], visual_df)

3830.63485188579

In [36]:
eeg_df = transform_eeg("./test_files/Rytle_J_02.xdf", exp_dt,"Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")
gps_df = transform_gps_xdf("./test_files/Rytle_J_02.xdf", exp_dt, "Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")
visual_df = transform_visual_distractions("./test_files/annotated.csv", exp_dt, "Rytle/2021/10/07/KZ/02/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf")

In [37]:
fact_df = create_fact_df(eeg_df, etd_df, gps_df)

In [34]:
visual_df

,start_lsl,end_lsl,is_visual,experiment_date,experiment_location_id,participant_id,experiment_nr


In [44]:
fact_df["eeg_joinable_column_with_etd_gps"].describe()

count    162128.000000
mean       5463.628109
std         187.206888
min        5139.418317
25%        5301.542562
50%        5463.645665
75%        5625.681789
max        5787.918815
Name: eeg_joinable_column_with_etd_gps, dtype: float64

In [38]:
def get_vis(row, visual_df):
    o = visual_df.loc[(visual_df["start_lsl"] <= row["eeg_joinable_column_with_etd_gps"]) & (row["eeg_joinable_column_with_etd_gps"] <= visual_df["end_lsl"])]["is_visual"]
    return True if len(o) else False

In [39]:
fact_df["is_visual"] = fact_df.apply(get_vis, args=(visual_df,), axis=1)

In [48]:
fact_df["is_visual"].value_counts()

False    155088
True       7040
Name: is_visual, dtype: int64

In [16]:
fact_df["velocity"].ddescribecribe()

count    185248.000000
mean          4.074095
std           4.547710
min           0.000000
25%           0.000000
50%           2.000000
75%           8.000000
max          16.000000
Name: velocity, dtype: float64

fact dataframe:   

| eeg | lsl_tst | etd | gps |
| -- | -- | -- | -- | 
| 2 | 6 | 7 | 2 |
| 12 | 16 | 8 | 3 |
| 6 | 10 | 9 | 3 |
| 7 | 11 | 10 | 3 |

* visual_df *

| start_lsl | end_lsl | is_visual |
| -- | -- | -- |
| 2 | 11 | True | 
| 12 | 16 | True | 

In [ ]:
eeg_etd_gps_visual = eeg_etd_gps.merge(
        visual_df[visual_columns],
        left_on="eeg_joinable_column_with_etd_gps",
        right_on="start_lsl",
        how="left",
        suffixes=("_eeg", "_etd_gps")
    )

In [22]:
def add(a, b = None):
    return a + (b  if b else 0)
    #return a + (b or 0)

In [23]:
add(4)

4

In [37]:
{} and True or None or [] or 0.0 or " " or "harish"

' '